Step 1 Load two image, define the mask position

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from scipy.sparse import csr_matrix, lil_matrix, linalg

src_img_path = "../data/astronaut.jpg"
dest_img_path = "../data/Apollo_15_flag_rover_LM_Irwin-1200x1200.jpg"

src_img = cv2.imread(src_img_path); src_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)
plt.imshow(src_img);plt.show()
dest_img = cv2.imread(dest_img_path); dest_img = cv2.cvtColor(dest_img, cv2.COLOR_BGR2RGB)
plt.imshow(dest_img);plt.show()

In [ ]:
cut_pos = [70, 280, 160, 285]
paste_pos = [400, 400+cut_pos[1]-cut_pos[0], 300, 300+cut_pos[3]-cut_pos[2]]
print(paste_pos)
cut_img = src_img[cut_pos[0]:cut_pos[1], cut_pos[2]:cut_pos[3], :]
plt.imshow(cut_img);plt.show()
cut_paste_img = np.zeros_like(dest_img)
cut_paste_img = dest_img
cut_paste_img[paste_pos[0]:paste_pos[1], paste_pos[2]:paste_pos[3], :] = cut_img
plt.imshow(cut_paste_img);plt.show()

Step 2 

In [ ]:
# def condition1(i, j, h, w):
#     if i == 0 and j == 0:
#         return True
#     elif i == 0 and j == w-1:
#         return True
#     elif i == h-1 and j == 0:
#         return True
#     elif i == h-1 and j == w-1:
#         return True
#     else:
#         return False
    
# def condition2(i, j, h, w):
#     if i == 0 and j <= w-2 and j >= 1:
#         return True
#     elif i == h-1 and j <= w-2 and j >= 1:
#         return True
#     elif i <= h-2 and i >= 1 and j == 0:
#         return True
#     elif i <= h-2 and i >= 1 and j == w-1:
#         return True
#     else:
#         return False

# def solve_poission_blending(cut_img, dest_img, paste_pos):
#     mask_h, mask_w = cut_img.shape[0], cut_img.shape[1]
#     offset_x = paste_pos[0]; offset_y = paste_pos[2]

#     A = np.zeros((mask_h*mask_w, mask_h*mask_w))
#     b = np.zeros((mask_h*mask_w, 3))

#     u = cv2.Sobel(cut_img, cv2.CV_64F, 1, 0, ksize=3)
#     v = cv2.Sobel(cut_img, cv2.CV_64F, 0, 1, ksize=3)
#     for i in range(mask_h):
#         for j in range(mask_w):
#             sha = i*mask_w+j
#             if condition1(i, j, mask_h, mask_w):
#                 continue
#             elif condition2(i, j, mask_h, mask_w):
#                 A[sha, sha] = 1
#                 b[sha, :] = dest_img[offset_x+i, offset_y+j, :]
#             else:
#                 A[sha, sha] = -4
#                 A[sha, sha-1] = 1
#                 A[sha, sha+1] = 1
#                 A[sha, (i-1)*mask_w+j] = 1
#                 A[sha, (i+1)*mask_w+j] = 1
#                 b[sha, :] = u[i+1-1, j-1, :]-u[i-1, j-1, :] + v[i-1, j+1-1, :]-v[i-1, j-1, :]
    
#     # Convert A to a CSR sparse matrix
#     A_sparse = csr_matrix(A)

#     # Reshaping b to a 2D array
#     b = b.reshape(-1, 3)

#     generate_img = np.zeros((mask_h, mask_w, 3))
#     for i in range(3):
#         # Solve the sparse linear equation
#         x = linalg.spsolve(A_sparse, b[:, i])
#         generate_img[:, :, i] = x.reshape(mask_h, mask_w)

#     return generate_img


# generate_img = solve_poission_blending(cut_img, dest_img, paste_pos)
# dest_img[paste_pos[0]:paste_pos[1], paste_pos[2]:paste_pos[3], :] = generate_img
# plt.imshow(dest_img);plt.show()

In [ ]:
def seamlessClone(src,dst,mask,center,flag):
    # 1. 求源图像的散度
    laplacian = cv2.Laplacian(np.float64(src),ddepth=-1,ksize=1)
    # 2. 计算mask区域的范围
    loc = np.nonzero(mask)
    xbegin, xend, ybegin, yend = np.min(loc[0]) - 1, np.max(loc[0]) + 1, np.min(loc[1]) - 1, np.max(loc[1]) + 1
    # 3. 根据mask区域范围截取出对应的mask、src、lap
    cutMask = mask[xbegin:xend, ybegin:yend]
    cutSrc = src[xbegin:xend, ybegin:yend]
    cutLap = laplacian[xbegin:xend, ybegin:yend]
    # 4. 为了方便后面计算复制的位置，将mask、laplacian、src变成和dst一样大
    finalMask = np.zeros((dst.shape[0], dst.shape[1]))  # 只复制长和宽，单通道
    finalSrc = np.zeros_like(dst)
    finalLap = np.zeros_like(dst, dtype=np.float64)
    # 5. 根据中心点，得到融合的x、y范围
    xmid = (np.max(loc[0]) - np.min(loc[0])) // 2 + 1
    xbegin = center[0] - xmid
    xend = center[0] + cutMask.shape[0] - xmid
    ymid = (np.max(loc[1]) - np.min(loc[1])) // 2 + 1
    ybegin = center[1] - ymid
    yend = center[1] + cutMask.shape[1] - ymid

    # 复制到对应位置
    finalMask[xbegin:xend, ybegin:yend] = cutMask
    print(xbegin, xend, ybegin, yend)
    finalSrc[xbegin:xend, ybegin:yend] = cutSrc
    finalLap[xbegin:xend, ybegin:yend] = cutLap

    # 逐通道求解
    result = [getX(a, finalMask, b) for a, b in zip(cv2.split(dst), cv2.split(finalLap))]
    
    # 合并三个通道
    final = cv2.merge(result)
    return final


def getX(dst, mask, lap):
    # 1 计算求解的像素个数
    loc = np.nonzero(mask)
    num = loc[0].shape[0] 
    # 2. 有多少个像素个数则需要多少个方程，需要构造num*num大小的稀疏矩阵和num大小的b向量
    # 3. A用lilmatrix，这种矩阵方便逐个添加
    A = lil_matrix((num, num), dtype=np.float64)
    b = np.ndarray((num, ), dtype=np.float64) # 一维数组,长度只有1, 而且最外层的长度为num
    # 4. 要将每个像素映射到0~num-1的索引之中，因为A系数矩阵也是根据索引求构造的
    hhash = {(x, y): i for i, (x, y) in enumerate(zip(loc[0], loc[1]))}
    # 用于找上下左右四个像素位置
    dx = [1,0,-1,0]
    dy = [0,1,0,-1]
    # 5. 构造A系数矩阵和b向量 
    for i, (x, y) in enumerate(zip(loc[0],loc[1])):
        A[i, i] = -4
        b[i] = lap[x, y]
        p = [(x + dx[j], y + dy[j]) for j in range(4)] # p为上下左右的像素位置
        for j in range(4):
            if p[j] in hhash: # 像素位置在mask内
                A[i, hhash[p[j]]] = 1
            else:
                b[i] -= dst[p[j]]
                
    # 由于A是稀疏矩阵，可以将lilmatrix转成cscmatrix，方便矩阵运算
    A = A.tocsc()
    # 6. 求解X
    X = linalg.splu(A).solve(b)

    # 7. 将X复制到对应位置
    result = np.copy(dst)
    for i, (x, y) in enumerate(zip(loc[0],loc[1])):
        if X[i] < 0:
            X[i] = 0
        if X[i] > 255:
            X[i] = 255
        result[x, y] = X[i]

    return result

mask = np.zeros((src_img.shape[0], src_img.shape[1]))
mask[cut_pos[0]:cut_pos[1], cut_pos[2]:cut_pos[3]] = 1
gb = seamlessClone(src_img, dest_img, mask, [500, 350], 0)
plt.imshow(gb)
plt.show()

In [1]:
def seamlessTiling(src):
    # 1. 融合的mask是整个图像矩形区域
    mask = np.ones((src.shape[0], src.shape[1]))
    # 2. 求出边界的位置
    boundary_mask = np.ones((src.shape[0], src.shape[1]))
    boundary_mask[np.ix_(np.arange(1, src.shape[0] - 1), np.arange(1, src.shape[1] - 1))] = 0
    _boundary = np.nonzero(boundary_mask)
    boundary = {(x, y): i for i, (x, y) in enumerate(zip(_boundary[0], _boundary[1]))}
    
    loc = np.nonzero(mask)
    num = loc[0].shape[0]  # 需要求解的像素个数
    A = lil_matrix((num, num), dtype=np.float64)
    hhash = {(x, y): i for i, (x, y) in enumerate(zip(loc[0], loc[1]))}
    # 找上下左右四个像素位置
    dx = [1, 0, -1, 0]
    dy = [0, 1, 0, -1]
    # 3. 逐通道求解A矩阵和b向量
    result = []
    for _src in cv2.split(src):
        # 3.1 求解原图散度
        # _lap = compute_laplacian(_src) 
        _lap = cv2.Laplacian(np.float64(_src),ddepth=-1,ksize=1)
        # 3.2 计算边界新的像素值
        tmp = np.zeros_like(_src)
        tmp[0] = (_src[0] + _src[-1]) * 0.5
        tmp[-1] = (_src[0] + _src[-1]) * 0.5
        tmp[:, 0] = (_src[:, 0] + _src[:, -1]) * 0.5
        tmp[:, -1] = (_src[:, 0] + _src[:, -1]) * 0.5
        b = np.ndarray((num,), dtype=np.float64)
        # 3.3 遍历图像每个像素
        for i, (x, y) in enumerate(zip(loc[0], loc[1])):
            # 如果像素是边界点，则b[i]为融合的边界值，A[i,i]=1，因此相当于新的图像的边界的像素值为融合的边界值
            if (x, y) in boundary:
                b[i] = tmp[x, y]
                A[i, i] = 1
            # 如果像素不是边界点，则b[i]为散度值，A[i,i] = 4，A[i,邻居] = 1
            else:
                b[i] = _lap[x, y]
                A[i, i] = -4
                p = [(x + dx[j], y + dy[j]) for j in range(4)]  # p为上下左右的像素位置
                for j in range(4):
                    A[i, hhash[p[j]]] = 1
        A = A.tocsr()
        # 3.4 求解像素值
        X = linalg.spsolve(A, b)
        # 3.5 将X复制到对应位置
        res = np.zeros_like(_src)
        for i, (x, y) in enumerate(zip(loc[0], loc[1])):
            if X[i] < 0:
                X[i] = 0
            if X[i] > 255:
                X[i] = 255
            res[x, y] = X[i]
        result.append(res)
    # 4. 合并三个通道
    final = cv2.merge(result)
    return final

src = cv2.imread("../data/kraft.png"); src = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
plt.imshow(src), plt.show()
result = seamlessTiling(src.astype(np.float64))
result_img = np.tile(result, (2, 2, 1))
cv2.imwrite("res.jpg",result_img)
ori_img = np.tile(src, (2, 2, 1))
cv2.imshow('orignal',ori_img)


NameError: name 'cv2' is not defined